In [1]:
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib.patches import Circle, Rectangle
from matplotlib.collections import PatchCollection
import numpy as np
import os
import re
from scipy import signal
from scipy import ndimage

%matplotlib

np.random.seed(77777)

Using matplotlib backend: Qt5Agg


In [2]:
def gen_template(w: float, h: float, dpi: int, loc: str, plot=False):
    '''
    Generate an image of a photogrammetry target.

    Parameters
    ----------
    w
        total image width, m
    h
        total image height, m
    dpi
        output image dpi, converts between matrix coordinates and real width
    loc
        [SW, NW, SE, NE, raft] generates a different pattern for each loc
    '''
    IN_TO_METERS = 0.0254
    w_in = w / IN_TO_METERS
    h_in = h / IN_TO_METERS
    w_px = np.round(w_in * dpi).astype(int)
    h_px = np.round(h_in * dpi).astype(int)

    fig, ax = plt.subplots()
    fig.set_size_inches(w_in, h_in)
    fig.tight_layout()
    fig.subplots_adjust(left=0., right=1., top=1., bottom=0.)
    ax.scatter([0, 0, w_px, w_px], [0, h_px, 0, h_px], color='k')

    # black background
    # ax.add_patch(Rectangle((0, 0), w_px, h_px, color='k'))
    pattern_size = 8
    pattern = np.random.random((pattern_size,pattern_size)) * 0.5 + .5
    pattern[0][0] = 0
    pattern[pattern_size - 1][pattern_size - 1] = 1
    ax.imshow(ndimage.zoom(pattern, h_px / pattern_size, order=0), cmap='Greys')

    # center cross
    c_size = dpi * 0.01 / IN_TO_METERS # 10 cm
    c_thick = dpi * 0.001 / IN_TO_METERS # 1 mm
    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 2), c_size, c_thick, color='w')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 2, h_px / 2 - c_size / 2), c_thick, c_size, color='w')) # vert

    circ_diam = c_size
    if loc == 'SW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4, h_px / 4),
        )
    elif loc == 'NW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'NE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'SE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, h_px / 4),
        )
    elif loc == 'raft':
        locs = (
            (w_px / 4, h_px / 4),
            (w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, h_px / 4),
        )
    else:
        raise ValueError(f'loc {loc} is not an option')

    for elem in locs:
        ax.add_patch(Circle(elem, circ_diam / 2, color='w', zorder=1))

    ax.set_aspect('equal')
    ax.set_xlim([0, w_px])
    ax.set_ylim([0, h_px])
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(f'{loc}_target.png')
    if not plot:
        plt.close()

plt.close('all')
locs = ['SW', 'NW', 'NE', 'SE', 'raft']
TARGET_W = 0.04 # 40 mm
TARGET_H = TARGET_W
# for loc in locs:
    # gen_template(TARGET_W, TARGET_H, 300, loc, plot=False)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [3]:
def measure_pos(filename, plot_corr_metrics=False):
    # locs = ['SW', 'NW', 'NE', 'SE', 'raft'] # all targets expected to be in the images
    locs = ['SW', 'raft'] # all targets expected to be in the images

    # The main figure to host the ground truth image, xcorr, and template
    f0 = plt.figure(constrained_layout=True)
    f0.suptitle(filename)
    axd = f0.subplot_mosaic(
        """
        ac
        ab
        """
    )
    # Ground truth
    im = image.imread(filename)
    im = np.sum(im.astype(float), axis=2) # single channel
    im = np.max(im) - im # invert value
    im /= im.max() # normalize
    im -= im.mean() # detrend
    stride = 5 # downsample for speed
    # im = ndimage.rotate(im, 90)
    im = im[::stride,::stride]
    axd['a'].imshow(im, cmap='Greys')
    axd['b'].set_title('Similarity Metric (xcorr)')
    axd['c'].set_title(f'Target Template Image')

    f1, ax = plt.subplots(nrows=len(locs))
    f1.suptitle(filename)
    f1.tight_layout()

    # record the best correlation position in pixels and the template scale factor, pixels per meter
    found_pos = []
    px_per_ms = []
    for i, loc in enumerate(locs):
        ax[i].set_title(loc)
        ax[i].set_xlabel('Template Image Size (px)')
        ax[i].set_ylabel('Correlation\nPeak-to-Sidelobe Ratio')
        l = None # placeholder for matplotlib line object

        tmp = image.imread(loc + '_target.png')
        tmp = np.sum(tmp.astype(float) , axis=2) # single channel
        tmp = np.max(tmp) - tmp # invert
        tmp /= tmp.max() # normalize
        tmp -= tmp.mean() # detrend
        # try different scales to find the size of the template, in pixels.
        # Proper range for this will depend on the angular size of the target in the image.
        pxs = np.logspace(7, 8, num=30, base=2) / stride
        sz = np.max(tmp.shape)
        zooms = pxs / sz
        psrs = []
        for j, zoom in enumerate(zooms):
            # scale template image
            tmp_zoom = ndimage.zoom(tmp, zoom)
            # axd['c'].imshow(tmp_zoom, cmap='Greys')
            # find best matching lags in image space
            xcorr = signal.fftconvolve(im, tmp_zoom[::-1, ::-1], mode='valid')
            # axd['b'].imshow(xcorr)
            # a metric for strength of the match
            sidelobe = np.partition(xcorr.flatten(), -2)[-2] # https://stackoverflow.com/q/33181350
            psr = np.max(xcorr) / sidelobe
            psrs.append(psr)
            # print(f'xcorr peak-to-sidelobe ratio: {psr}')
            if not l:
                l, = ax[i].plot(pxs[:len(psrs)], psrs)
            else:
                l.set_data(pxs[:len(psrs)], psrs)
                ax[i].set_xlim(.9 * min(pxs[:len(psrs)]), max(pxs[:len(psrs)]) * 1.1)
                ax[i].set_ylim(.9 * min(psrs), max(psrs) * 1.1)
            
            f0.canvas.draw_idle()
            f1.canvas.draw_idle()
            plt.pause(0.001)

        # Generally, it seems a match has been found when the PSR reaches a global maximum.
        best_idx = np.argmax(psrs)
        # recover zoomed template
        tmp_zoom = ndimage.zoom(tmp, zooms[best_idx])
        axd['c'].imshow(tmp_zoom, cmap='Greys')
        # recover xcorr
        xcorr = signal.fftconvolve(im, tmp_zoom[::-1, ::-1], mode='valid')
        axd['b'].imshow(xcorr)
        # the indices of the best match lags
        y,x = np.unravel_index(np.argmax(xcorr), xcorr.shape)
        x_cent = x + tmp_zoom.shape[0] / 2
        y_cent = y + tmp_zoom.shape[1] / 2
        print(f'{loc} target center found at {(x_cent * stride, y_cent * stride)} px in image')
        axd['a'].add_patch(Rectangle((x, y), tmp_zoom.shape[0], tmp_zoom.shape[1], fill=False, color='red'))
        axd['a'].scatter(x_cent, y_cent)
        axd['a'].annotate(f'{loc}, est. size = {pxs[best_idx]:.2f} px', (x,y))
        # Ignore the raft target, as rotations and projection effects are more likely to affect the size estimate
        if loc != 'raft':
            px_per_ms.append(pxs[best_idx] * stride / (TARGET_W))
        found_pos.append((x_cent * stride, y_cent * stride))
        # print(f'Calculated pixels per m: {pxs[best_idx] * stride / (TARGET_W)}')
                
        if loc == 'raft':
            # Go on to try to find the best rotation
            im_crop = im[y:y + tmp_zoom.shape[1], x:x + tmp_zoom.shape[0]]
            rot_psrs = []
            angs = np.arange(-10, 10, .1)
            for ang in angs:
                im_crop_rot = ndimage.rotate(im_crop, ang, reshape=False)
                rot_xcorr = signal.fftconvolve(im_crop_rot, tmp_zoom[::-1, ::-1], mode='valid')
                rot_sidelobe = np.partition(xcorr.flatten(), -2)[-2] # https://stackoverflow.com/q/33181350
                rot_psrs.append(np.max(rot_xcorr) / rot_sidelobe)
            best_rot = angs[np.argmax(rot_psrs)]
            # print('Raft rotation:', best_rot)
            axd['c'].set_title(f'Raft target after derotation by {best_rot:.3f} deg:')
            axd['c'].imshow(ndimage.rotate(im_crop, best_rot, reshape=False))

    file_base = os.path.splitext(os.path.basename(filename))[0]
    f0.savefig(f'{file_base}_found_locs.png')
    f1.savefig(f'{file_base}_psrs.png')
    plt.pause(1)
    plt.close(f0)
    plt.close(f1)

    # which position is the raft in?
    sw_loc = np.array(found_pos[0])
    raft_loc = np.array(found_pos[-1])

    return px_per_ms, sw_loc, raft_loc, best_rot

In [4]:
def create_files_in_order(dataset_name, xs, ys):
    '''
    Make copies of and rename files according to their position in the error map grid.
    Assumed that photo filenames are ordered such that they depict each position in
    increasing rows of x, increasing columns of y, row-major.
    '''
    from shutil import copyfile
    import glob

    orig_photos = sorted(glob.glob(os.path.join('img', 'orig', dataset_name, '*.JPG')))
    count = 0
    for y_elem in ys:
        for x_elem in xs:
            fname = 'x' + str(np.round(x_elem * 1000, 2)).rstrip('0').rstrip('.') + 'y' + str(np.round(y_elem * 1000, 2)).rstrip('0').rstrip('.') + '.jpeg'
            # print(x_elem, y_elem, orig_photos[count], os.path.join('img', dataset_name, fname))
            copyfile(orig_photos[count], os.path.join('img', dataset_name, fname))
            count += 1

In [5]:
dataset_name = 'point_to_point'
xs = np.arange(.15, .55, .05)#[::2] # point_to_point (max coverage scan)
ys = np.arange(.15, .55, .05)#[::2]
# dataset_name = '5x5_raster_coarse_K3_v2' # coarse raster scan
# xs = np.linspace(.3, .45, num=5)
# ys = np.linspace(.2, .35, num=5)
# create_files_in_order(dataset_name, xs, ys)

In [6]:
X,Y = np.meshgrid(xs, ys)
meas_sw_pos_px_x = np.zeros(X.shape)
meas_sw_pos_px_y = np.zeros(Y.shape)
meas_raft_pos_px_x = np.zeros(X.shape)
meas_raft_pos_px_y = np.zeros(Y.shape)
x_err = np.zeros(X.shape)
y_err = np.zeros(Y.shape)
raft_ang = np.zeros(Y.shape)
px_per_ms = []
for i, x_elem in enumerate(xs):
    for j, y_elem in enumerate(ys):
        # create filename from query points
        fname = 'x' + str(np.round(x_elem * 1000, 2)).rstrip('0').rstrip('.') + 'y' + str(np.round(y_elem * 1000, 2)).rstrip('0').rstrip('.') + '.jpeg'
        print(f'Processing {fname}')
        path = os.path.join('img', dataset_name, fname)
        if os.path.exists(path):
            px_per_m, sw_pos, raft_pos, ang = measure_pos(path)
            px_per_ms += px_per_m
            meas_sw_pos_px_x[i][j] = sw_pos[0]
            meas_sw_pos_px_y[i][j] = sw_pos[1]
            meas_raft_pos_px_x[i][j] = raft_pos[0]
            meas_raft_pos_px_y[i][j] = raft_pos[1]
            raft_ang[i][j] = ang

Processing x150y150.jpeg
SW target center found at (1237.5, 3017.5) px in image
raft target center found at (1875.0, 2440.0) px in image
Processing x150y200.jpeg
SW target center found at (1262.5, 3007.5) px in image
raft target center found at (1892.5, 2217.5) px in image
Processing x150y250.jpeg
SW target center found at (1257.5, 3012.5) px in image
raft target center found at (1897.5, 2032.5) px in image
Processing x150y300.jpeg
SW target center found at (1257.5, 3007.5) px in image
raft target center found at (1882.5, 1822.5) px in image
Processing x150y350.jpeg
SW target center found at (1242.5, 3012.5) px in image
raft target center found at (1857.5, 1622.5) px in image
Processing x150y400.jpeg
SW target center found at (1242.5, 3012.5) px in image
raft target center found at (1850.0, 1420.0) px in image
Processing x150y450.jpeg
SW target center found at (1240.0, 2990.0) px in image
raft target center found at (1840.0, 1200.0) px in image
Processing x150y500.jpeg
SW target center

In [19]:
# remove outliers due to failed position finding, if necessary (replace with nans)
px_per_ms_clean = px_per_ms[:]
print(px_per_ms_clean)
ax = plt.axes()
ax.hist(px_per_ms_clean)
print(np.logspace(6.8, 7.5, num=20, base=2) / TARGET_W)
meas_sw_pos_px_x_clean = meas_sw_pos_px_x[:]
meas_sw_pos_px_y_clean = meas_sw_pos_px_y[:]
meas_raft_pos_px_x_clean = meas_raft_pos_px_x[:]
meas_raft_pos_px_x_clean[1][0] = np.nan
meas_raft_pos_px_x_clean[4][0] = np.nan
meas_raft_pos_px_y_clean = meas_raft_pos_px_y[:]
raft_ang_clean = raft_ang[:]

[4063.9974343492568, 3874.2954430775376, 4063.9974343492568, 4063.9974343492568, 4063.9974343492568, 4063.9974343492568, 3968.0129461202796, 3874.2954430775376, 3968.0129461202796, 3874.2954430775376, 4063.9974343492568, 4063.9974343492568, 4063.9974343492568, 3874.2954430775376, 4063.9974343492568, 4063.9974343492568, 3968.0129461202796, 3968.0129461202796, 4063.9974343492568, 3874.2954430775376, 4063.9974343492568, 4063.9974343492568, 4063.9974343492568, 3968.0129461202796, 4063.9974343492568, 3968.0129461202796, 3874.2954430775376, 3874.2954430775376, 4063.9974343492568, 4063.9974343492568, 3968.0129461202796, 3874.2954430775376, 3968.0129461202796, 4063.9974343492568, 3874.2954430775376, 3874.2954430775376, 4063.9974343492568, 3874.2954430775376, 4063.9974343492568, 3968.0129461202796, 3874.2954430775376, 4063.9974343492568, 3968.0129461202796, 3874.2954430775376, 3968.0129461202796, 4063.9974343492568, 4063.9974343492568, 4063.9974343492568, 3693.4484882751576, 3968.0129461202796,

In [20]:
# average all best-corr target estimates of pixels per meter to find the image scale
mean_px_per_m = np.nanmean(px_per_ms_clean)
std_px_per_m = np.nanstd(px_per_ms_clean)
print('Mean pixels per meter:', mean_px_per_m)
print('Pixel per meter std. dev.:', std_px_per_m)
print('Error in difference of two pixel coordinates:', 2. ** 0.5 * std_px_per_m, 'pixels per meter of distance')
# Convert image coordinates to meters and find raft positions relative to SW in each image
raft_x_mirror_coords = (meas_raft_pos_px_x_clean - meas_sw_pos_px_x_clean) / mean_px_per_m
raft_y_mirror_coords = (meas_sw_pos_px_y_clean - meas_raft_pos_px_y_clean) / mean_px_per_m
# Find position errors relative to commands
x_err_clean = X.T - raft_x_mirror_coords
y_err_clean = Y.T - raft_y_mirror_coords


Mean pixels per meter: 3976.4968870235343
Pixel per meter std. dev.: 91.46753678869682
Error in difference of two pixel coordinates: 129.35463104343506 pixels per meter of distance


In [23]:
f2, axs = plt.subplots(ncols=2, squeeze=False, figsize=(16,4))
title = dataset_name + ' Error Map'
plt.suptitle(title)
im_x = axs[0][0].contourf(X.T, Y.T, x_err_clean)
cbar = plt.colorbar(im_x, ax=axs[0][0])
cbar.set_label('Error Magnitude (m)')
axs[0][0].set_title('X Error (m)')
axs[0][0].set_xlabel('Commanded x-Position (m)')
axs[0][0].set_ylabel('Commanded y-Position (m)')

print('Max X Error:', np.nanmax(x_err_clean), np.unravel_index(x_err_clean.argmax(), x_err_clean.shape))
print('Min X Error:', np.nanmin(x_err_clean))

im_y = axs[0][1].contourf(X.T, Y.T, y_err_clean)
cbar = plt.colorbar(im_y, ax=axs[0][1])
cbar.set_label('Error Magnitude (m)')
axs[0][1].set_title('Y Error (m)')
axs[0][1].set_xlabel('Commanded x-Position (m)') 
axs[0][1].set_ylabel('Commanded y-Position (m)')

print('Max Y Error:', np.nanmax(y_err_clean), np.unravel_index(y_err_clean.argmax(), y_err_clean.shape))
print('Min Y Error:', np.nanmin(y_err_clean))

f2.savefig(os.path.join('output', dataset_name, title + '.png'), dpi=300)

Max X Error: 0.015905568471124842 (1, 0)
Min X Error: -0.08882845871059075
Max Y Error: 0.45691563473612684 (5, 6)
Min Y Error: -0.018119197861832304


In [22]:
f3, axes = plt.subplots()
title = dataset_name + ' Error Magnitude'
plt.suptitle(title)
err_mag = np.sqrt(x_err_clean ** 2. + y_err_clean ** 2.)
im = axes.contourf(X.T, Y.T, err_mag)
cbar = plt.colorbar(im, ax=axes)
cbar.set_label('Error Magnitude (m)')
axes.set_xlabel('Commanded x-Position (m)')
axes.set_ylabel('Commanded y-Position (m)')

f3.savefig(os.path.join('output', dataset_name, title + '.png'), dpi=300)

print('Max error magnitude:', np.nanmax(err_mag))
print('Min error magnitude:', np.nanmin(err_mag))
print('Error magnitude mean:', np.nanmean(err_mag))
print('Error magnitude std:', np.nanstd(err_mag))

Max error magnitude: 0.4585096719646702
Min error magnitude: 0.000898347465936991
Error magnitude mean: 0.023285002913099952
Error magnitude std: 0.07353112936330941


In [11]:
f4, axes = plt.subplots()
title = dataset_name + ' Raft Angular Offset'
plt.suptitle(title)
im = axes.contourf(X.T, Y.T, raft_ang_clean)
cbar = plt.colorbar(im, ax=axes)
cbar.set_label('Raft Angle (deg)')
axes.set_xlabel('Commanded x-Position (m)')
axes.set_ylabel('Commanded y-Position (m)')

f4.savefig(os.path.join('output', dataset_name, title + '.png'), dpi=300)

print('Max angle:', np.nanmax(raft_ang_clean))
print('Min angle:', np.nanmin(raft_ang_clean))
print('Mean angle:', np.nanmean(raft_ang_clean))
print('Std angle:', np.nanstd(raft_ang_clean))

Max angle: 8.399999999999935
Min angle: -8.700000000000005
Mean angle: -0.8640625000000324
Std angle: 3.8422961554380013


In [12]:
f5, axes = plt.subplots()
axes.set_facecolor('gainsboro')
title = dataset_name + ' Commanded vs. Actual Positions'
plt.suptitle(title)
axes.scatter(X.T, Y.T, facecolor='none', color='k', label='Commanded')
axes.scatter(raft_x_mirror_coords, raft_y_mirror_coords, c=err_mag, label='Measured')
# https://stackoverflow.com/a/64546653
axes.quiver(
    raft_x_mirror_coords, raft_y_mirror_coords,
    -x_err_clean, -y_err_clean,
    err_mag,
    pivot='tip',
    angles='xy',
    scale_units='xy',
    scale=1,
    headwidth=2.5,
    headlength=4,
    edgecolors='k',
    linewidth=.5,
    width=4e-3
)
axes.set_xticks(xs)
axes.set_yticks(ys)
axes.set_aspect('equal')
axes.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
axes.grid(True, color='k', linestyle='--')
axes.set_xlabel('x-Position (m)')
axes.set_ylabel('y-Position (m)')
f5.tight_layout()

f5.savefig(os.path.join('output', dataset_name, title + '.png'), dpi=300)


In [13]:
plt.close('all')